In [4]:
import ee
import pprint

# Initialiser Earth Engine
ee.Authenticate()
ee.Initialize(project = "ndvi-460519")

# Définir la zone d'étude (exemple polygon)
geometry = ee.Geometry.Polygon([
            [
              [
                -7.580227249237301,
                32.71192977492841
              ],
              [
                -7.579668197542941,
                32.71273711144024
              ],
              [
                -7.579430381562711,
                32.71297082081924
              ],
              [
                -7.578395389554159,
                32.712623726645184
              ],
              [
                -7.578495048223404,
                32.712080578721284
              ],
              [
                -7.577652595179951,
                32.71189509679929
              ],
              [
                -7.577701726398622,
                32.71125791490734
              ],
              [
                -7.5784477793902765,
                32.71137083285161
              ],
              [
                -7.578440455000989,
                32.711832561224256
              ],
              [
                -7.578513721216516,
                32.71183654784859
              ],
              [
                -7.578524972494178,
                32.71187919935686
              ],
              [
                -7.578725210611426,
                32.71184994300518
              ],
              [
                -7.5791986959597955,
                32.71172244833306
              ],
              [
                -7.579401488946102,
                32.71169997554632
              ],
              [
                -7.579579884945278,
                32.71170424796387
              ],
              [
                -7.579770848272175,
                32.71175595331752
              ],
              [
                -7.580228935592438,
                32.711927779939785
              ],
              [
                -7.580227249237301,
                32.71192977492841
              ]
            ]
          ])

# Charger image Sentinel-2
image = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(geometry) \
    .filterDate('2024-05-01', '2024-06-01') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median()

# === Fonctions d'indices ===
def compute_indices(img):
    L = 0.5
    indices = {
        'NDVI': img.normalizedDifference(['B8', 'B4']),
        'EVI': img.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
                              {'NIR': img.select('B8'), 'RED': img.select('B4'), 'BLUE': img.select('B2')}),
        'SAVI': img.expression('((NIR - RED) / (NIR + RED + L)) * (1 + L)',
                               {'NIR': img.select('B8'), 'RED': img.select('B4'), 'L': ee.Number(L)}),
        'MSAVI': img.expression('((2 * NIR + 1) - sqrt((2 * NIR + 1) ** 2 - 8 * (NIR - RED))) / 2',
                                {'NIR': img.select('B8'), 'RED': img.select('B4')}),
        'MSAVI2': img.expression('(2 * NIR + 1 - sqrt((2 * NIR + 1) ** 2 - 8 * (NIR - RED))) / 2',
                                 {'NIR': img.select('B8'), 'RED': img.select('B4')}),
        'PRI': img.expression('(RE1 - RE2) / (RE1 + RE2)',
                              {'RE1': img.select('B5'), 'RE2': img.select('B6')}),
        'CGI': img.normalizedDifference(['B7', 'B5']),
        'NDWI': img.normalizedDifference(['B3', 'B8']),
        'NDRE': img.normalizedDifference(['B8', 'B5']),
    }
    return indices

# === Plages par indice ===
breaks_dict = {
    'NDVI': [0.2, 0.4, 0.6, 0.8],
    'EVI': [0.2, 0.4, 0.6, 0.8],
    'SAVI': [0.2, 0.4, 0.6, 0.8],
    'MSAVI': [0.2, 0.4, 0.6, 0.8],
    'MSAVI2': [0.2, 0.4, 0.6, 0.8],
    'PRI': [0.02, 0.04, 0.06],
    'CGI': [0.2, 0.4, 0.6],
    'NDWI': [0.0, 0.2, 0.4],
    'NDRE': [0.2, 0.3, 0.4]
}

# === Fonction de classification ===
def classify_index(index_img, breaks):
    expr = ""
    for i, b in enumerate(breaks):
        expr += f"(b <= {b}) ? {i + 1} : "
    expr += f"{len(breaks) + 1}"
    return index_img.expression(expr, {'b': index_img}).rename('class')

# === Calculer et afficher les stats ===
indices = compute_indices(image)

print("\n=== Résultats par indice ===\n")

for name, index_img in indices.items():
    breaks = breaks_dict[name]
    class_img = classify_index(index_img, breaks)
    histo = class_img.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=geometry,
        scale=10,
        maxPixels=1e9
    )
    values = histo.get('class').getInfo()

    print(f"--- {name} ---")
    for k in sorted(values.keys(), key=int):
        print(f"Classe {k}: {values[k]} pixels")
    print()



=== Résultats par indice ===

--- NDVI ---
Classe 1: 9.525490196078431 pixels
Classe 2: 157.87450980392163 pixels
Classe 3: 89.79607843137256 pixels

--- EVI ---
Classe 1: 0.36470588235294116 pixels
Classe 2: 20.956862745098036 pixels
Classe 3: 72.65490196078431 pixels
Classe 4: 105.44313725490196 pixels
Classe 5: 57.77647058823529 pixels

--- SAVI ---
Classe 1: 0.403921568627451 pixels
Classe 2: 36.396078431372544 pixels
Classe 3: 130.6 pixels
Classe 4: 83.78039215686276 pixels
Classe 5: 6.015686274509804 pixels

--- MSAVI ---
Classe 1: 0.0392156862745098 pixels
Classe 2: 26.188235294117646 pixels
Classe 3: 178.78431372549028 pixels
Classe 4: 52.18431372549019 pixels

--- MSAVI2 ---
Classe 1: 0.0392156862745098 pixels
Classe 2: 26.188235294117646 pixels
Classe 3: 178.78431372549028 pixels
Classe 4: 52.18431372549019 pixels

--- PRI ---
Classe 1: 257.19607843137265 pixels

--- CGI ---
Classe 1: 100.66666666666666 pixels
Classe 2: 156.52941176470586 pixels

--- NDWI ---
Classe 1: 257.1